In [3]:
import keras

Using TensorFlow backend.


In [4]:
import pandas as pd
import numpy as np
import glob

In [5]:
from utils.schemas import schema_train_4


# Load Data

Para cargar los datos con pandas, voy a usar la misma forma que usamos en LightGBM

In [6]:
!ls ../../../data/

data_4.zip  df_num    test_final_4  train_final_4
df_cat	    test.csv  train.csv     train_model_little.csv


In [7]:
RUTA_TRAIN = "../../../data/train_model_little.csv"
RUTA_TEST = ""

In [86]:
# print('Cargando datos del TRAIN')
# path = RUTA_TRAIN
# allFiles = glob.glob(path + "/*.csv")
# list_ = []
# for file_ in allFiles:
#     df = pd.read_csv(file_)
#     df = (df.fillna(-1)).astype(schema_train_4)
#     list_.append(df)

# train = pd.concat(list_, axis = 0, ignore_index= True)

# train.head()

In [9]:
df = pd.read_csv(RUTA_TRAIN)
df = (df.fillna(-1)).astype(schema_train_4)

train = df

train.head()

,MachineIdentifier,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,...,count(DISTINCT AvSigVersion_Type),count(DISTINCT AvSigVersion_AlertLevel),count1,count2,count3,count4,count5,count6,count7,count8
0,000039104d7f42c15fdc0d1bcb9d4b4a,0,7,0,1950,53447,1,1,1,43,...,2,1,338532,3073400,717902,330725,113,7586007,1,8
1,00016c9d7670346f6d2f256c0091745a,0,7,0,0,53447,1,1,1,201,...,-1,-1,1787574,225416,8449217,242216,133,7586007,4641,5
2,00017fa35ed739442dfc8787ef8d8182,0,7,0,0,53447,1,1,1,60,...,1,1,507412,8215768,8449217,1099043,1,7586007,3860,18
3,0002cd8b6889b452fcf5b2438056f237,0,7,0,0,53447,1,1,1,158,...,2,1,228218,922300,8449217,1734631,534,7586007,1352,20
4,0004eb91add58287df74f019e3074e28,0,7,0,0,53447,1,1,1,44,...,2,2,228218,8215768,8449217,1884268,51,7586007,397,327


## Generate X and Y 

In [10]:
drop_version = ['AvSigVersion_index', 'EngineVersion_index', 'Census_OSVersion_index', 'AppVersion_index']

sel_cols = [c for c in train.columns if c not in ['MachineIdentifier',
                                                      'HasDetections',
                                                      'Census_DeviceFamily_Windows.Server',
                                                      'Census_DeviceFamily_Windows.Desktop'
                                                     ]+drop_version]

In [11]:
X = train.loc[:, sel_cols]
y = train.loc[:,'HasDetections']

## Train y test

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [15]:
print("X_train :"+str(X_train.shape))

print("X_test :"+str(X_test.shape))


print("y_train :"+str(y_train.shape))

print("y_test :"+str(y_test.shape))

X_train :(89188, 117)
X_test :(22298, 117)
y_train :(89188,)
y_test :(22298,)


# Keras Model

In [16]:


from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Embedding,Input,Concatenate,Reshape

# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import cross_val_score
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline

In [17]:
dimension = X_train.shape[1]

In [18]:
dimension

117

## Metrics

### AUC 1

Easy

In [ ]:
from sklearn.metrics import roc_auc_score
import tensorflow as tf

def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

### AUC 2

For Keras.

A little more complex

In [ ]:
from keras import callbacks
from sklearn.metrics import roc_auc_score

class printAUC(callbacks.Callback):
    def __init__(self, X_train, y_train):
        super(printAUC, self).__init__()
        self.bestAUC = 0
        self.X_train = X_train
        self.y_train = y_train
        
    def on_epoch_end(self, epoch, logs={}):
        pred = self.model.predict(np.array(self.X_train))
        auc = roc_auc_score(self.y_train, pred)
        print("Train AUC: " + str(auc))
        pred = self.model.predict(self.validation_data[0])
        auc = roc_auc_score(self.validation_data[1], pred)
        print ("Validation AUC: " + str(auc))
        if (self.bestAUC < auc) :
            self.bestAUC = auc
            self.model.save("bestNet.h5", overwrite=True)
        return

## EarlyStopping

In [ ]:
earlyStopping = keras.callbacks.EarlyStopping(monitor='auroc', patience=20, verbose=2,min_delta=0.01, mode='auto')

# Model work



## Embedding Layers

Primero, identificamos las variables categoricas

In [19]:
categoricas = ['ProductName_index','EngineVersion_0_index','EngineVersion_1_index','Census_OSVersion_0_index','Census_OSVersion_1_index','AppVersion_0_index','AppVersion_1_index','AvSigVersion_0_index','AvSigVersion_1_index','OsVer_0_index','OsVer_1_index','Platform_index','Processor_index','OsPlatformSubRelease_index','OsBuildLab_index','SkuEdition_index','PuaMode_index','SmartScreen_index','Census_MDC2FormFactor_index','Census_DeviceFamily_index','Census_ProcessorClass_index','Census_PrimaryDiskTypeName_index','Census_ChassisTypeName_index','Census_PowerPlatformRoleName_index','Census_InternalBatteryType_index','Census_OSArchitecture_index','Census_OSBranch_index','Census_OSEdition_index','Census_OSSkuName_index','Census_OSInstallTypeName_index','Census_OSWUAutoUpdateOptionsName_index','Census_GenuineStateName_index','Census_ActivationChannel_index','Census_FlightRing_index','OsVer_index','DefaultBrowsersIdentifier','AVProductStatesIdentifier','CountryIdentifier','CityIdentifier','OrganizationIdentifier','GeoNameIdentifier','LocaleEnglishNameIdentifier','IeVerIdentifier','Census_OEMNameIdentifier','Census_OEMModelIdentifier','Census_ProcessorManufacturerIdentifier','Census_ProcessorModelIdentifier','Census_OSInstallLanguageIdentifier','Census_OSUILocaleIdentifier','Census_FirmwareManufacturerIdentifier','Census_FirmwareVersionIdentifier','Wdft_RegionIdentifier','RtpStateBitfield','prediction_2','prediction_4','prediction_8','prediction_16','prediction_32','prediction_64']

booleanas = ['Census_IsSecureBootEnabled','Census_IsWIMBootEnabled','Census_IsVirtualDevice','Census_IsTouchEnabled','Census_IsPenCapable','Census_IsAlwaysOnAlwaysConnectedCapable','Wdft_IsGamer','Census_IsPortableOperatingSystem','Census_IsFlightingInternal','Census_IsFlightsDisabled','IsProtected','IsBeta','IsSxsPassiveMode','AVProductsInstalled','AVProductsEnabled','HasTpm','Census_HasOpticalDiskDrive','Census_ThresholdOptIn','Census_InternalBatteryType_informed','Firewall','SMode','AutoSampleOptIn']

numericas = ['OsBuild','OsSuite','UacLuaenable','Census_ProcessorCoreCount','Census_PrimaryDiskTotalCapacity','Census_SystemVolumeTotalCapacity','Census_TotalPhysicalRAM','Census_InternalPrimaryDiagonalDisplaySizeInInches','Census_InternalPrimaryDisplayResolutionHorizontal','Census_InternalPrimaryDisplayResolutionVertical','Census_InternalBatteryNumberOfCharges','Census_OSBuildNumber','Census_OSBuildRevision','OsBuildLab_diff','std_diff_DateOsBuildLab','AvSigVersion_diff','std_diff_AvSigVersion','OSVersion_diff','std_diff_OSVersion','max_OsBuildLab_diff','max_AvSigVersion_diff','max_OSVersion_diff','ratio_OsBuildLab_diff','ratio_AvSigVersion_diff','ratio_OSVersion_diff','count(DISTINCT AvSigVersion_Name)','count(DISTINCT AvSigVersion_Type)','count(DISTINCT AvSigVersion_AlertLevel)','count1','count2','count3','count4','count5','count6','count7','count8']


```py
input_numeric = Input(shape=(24,))
embedding_numeric = Dense(16)(input_numeric) 
inputs.append(input_numeric)
embeddings.append(embedding_numeric)

x = Concatenate()(embeddings)
x = Dense(80, activation='relu')(x)
x = Dropout(.35)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(.15)(x)
x = Dense(10, activation='relu')(x)
x = Dropout(.15)(x)
output = Dense(1, activation='sigmoid')(x)
```

In [80]:
ins = []
outs = []

In [81]:
for i in categoricas:
    x = Input(shape=(1,))
    ins.append(x)
    x = Embedding(df[i].nunique(), df[i].nunique(), input_length=1)(x)
    x = Reshape(target_shape=(df[i].nunique(),))(x)
    outs.append(x)
    
# for i in booleanas:
#     x = Input(shape=(1,))
#     ins.append(x)   
#     x = Embedding(df[i].nunique(), df[i].nunique(), input_length=1)(x)
#     x = Reshape(target_shape=(df[i].nunique(),))(x)
#     outs.append(x)

## Network Architecture

In [82]:
len(numericas+categoricas)

95

In [83]:
primer_input_len = len(numericas+booleanas)
primer_input_len

58

In [84]:
x = Input(shape=(primer_input_len, ))
ins.append(x)
x = Dense(50)(x) 
outs.append(x)

x = Concatenate()(outs)
#then continue making NN
x = Dense(100, activation='relu')(x)
x = Dense(100, activation='relu')(x)
# x = Dense(1, activation='sigmoid')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=ins, outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam')
#model.fit( [X_train[col] for col in ListCategorical] + [X_train[ListNonCategorical]], 

In [75]:
len(categoricas)+58

117

In [85]:
model.fit( [X_train[col] for col in categoricas] +  [X_train[numericas+booleanas]], 
           y_train,
          epochs=epochs_to_train)

Instructions for updating:
Use tf.cast instead.


/home/rde/stuff_workspace/2_Kaggle/kaggle_microsoft/enviroment/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:107: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 285103225 elements. This may consume a large amount of memory.
  num_elements)
/home/rde/stuff_workspace/2_Kaggle/kaggle_microsoft/enviroment/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:107: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 295977616 elements. This may consume a large amount of memory.
  num_elements)
/home/rde/stuff_workspace/2_Kaggle/kaggle_microsoft/enviroment/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:107: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 171845881 elements. This may consume a large amount of memory.
  num_elements)


Epoch 1/15


InvalidArgumentError: indices[2,0] = 2 is not in [0, 2)
	 [[{{node embedding_173/embedding_lookup}}]]

In [ ]:
model.fit( [X_train[col] for col in categoricas] + [X_train[col] for col in booleanas] +  [X_train[numericas]], 
           y_train,
          epochs=epochs_to_train)

In [ ]:
model.fit(X_train, y_train,
          epochs=epochs_to_train,
          batch_size=len_batch_size,
            validation_data = (X_test,y_test))

# Ejemplo de Modelos

Este es un "esquema" que me dijo un chaval __TODO: PONER EL NOMBRE__ para hacer los embeddings.

```py
x = Input(shape=(len(ListNonCategorical), ))
ins.append(x)
outs.append(x)

x = Concatenate(outs)
#then continue making NN
x = Dense(100)(x)
x = Dense(100)(x)
x = Dense(1)(x)
```

# Old Model

Este modelo ya no se usa.

In [ ]:
model = Sequential()

model.add(Dense(256, input_dim=dimension, kernel_initializer='uniform', activation='relu'))
model.add(Dense(512))
model.add(Dense(128))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam',metrics = ['acc'])

model.summary()

## Fit Model

In [ ]:
len_batch_size = int(X_train.shape[0]/1000)
epochs_to_train = 15

In [ ]:
model.fit(X_train, y_train,
          epochs=epochs_to_train,
          batch_size=len_batch_size,
            validation_data = (X_test,y_test))
#              callbacks=[earlyStopping])

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=int(X_train.shape[0]/100),verbose=1)

In [ ]:
score

# Graphs

In [ ]:
model.history.history.keys()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(model.history.history['loss'],label="Loss")
plt.plot(model.history.history['auroc'],label="acc")
plt.legend()

In [ ]:
plt.plot(model.history.history['acc'])

In [ ]:
plt.plot(model.history.history['auroc'])

# Utils

## Clean Memory

In [ ]:
del model

In [ ]:
import gc
gc.collect()

# Prediction

In [ ]:
print('Cargando datos del TEST')
path = '../../data/test_final_3'
allFiles = glob.glob(path + "/*.csv")
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_)
    df = (df.fillna(-1)).astype(schema_train_3)
    list_.append(df)


test = pd.concat(list_, axis = 0, ignore_index = True).fillna(-1)

sel_cols = [c for c in test.columns if c not in ['MachineIdentifier',
                                                 'HasDetections',
                                                 'Census_DeviceFamily_Windows.Server',
                                                 'Census_DeviceFamily_Windows.Desktop'
                                                 ]
            ]

X_test = test.loc[:, sel_cols]

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions

In [ ]:
predf= pd.DataFrame(predictions,columns=['HasDetections'])

In [ ]:
pd.concat([X_machines,predf],axis=1).to_csv('keras_model.csv', index=None)

In [ ]:
X_machines = test.loc[:, 'MachineIdentifier']


df_prds = pd.DataFrame({'MachineIdentifier': X_machines, 'HasDetections': predictions})

df_prds.to_csv('keras_model.csv', index=None)

In [ ]:
del predictions

### Save train in one file

In [ ]:
train.to_csv("train_uniq.csv")

In [ ]:
test.to_csv("test_uniq.csv")